In [1]:
from numpy import sqrt
def assemble_network(load_const = 10.0, transfer_const = 1.0, 
               elongation_const = 100.0, nucleation_const = 1.0, capping_const = 0.1, 
               ppr_max = 1.0, wh2_max = 1.0):
    wh2_eq = sqrt(capping_const / nucleation_const)
    # Set up quadratic.
    a = elongation_const * (capping_const + elongation_const * wh2_eq)
    b = (load_const + transfer_const * (wh2_max - wh2_eq)) * (capping_const + elongation_const * wh2_eq)
    c = - load_const * transfer_const * (wh2_max - wh2_eq) * ppr_max
    if (b**2 - 4 * a * c < 0):
        print("Non-real solution")
    ends_eq = (-b + sqrt(b**2 - 4 * a * c)) / (2 * a)
    if ends_eq < 0.0:
        ends_eq = (-b - sqrt(b**2 - 4 * a * c)) / (2 * a)
    if ends_eq < 0.0:
        print("Network failed to assemble")
    ppr_eq = load_const * ppr_max / (load_const + elongation_const * ends_eq + transfer_const * (wh2_max - wh2_eq))
    return ppr_eq, wh2_eq, ends_eq

In [2]:
from numpy import linspace, zeros
def capping_effect(k_cap_max, k_load, k_transfer, k_pol, k_nuc, pa_max, wa_max):
    capping_const_row = linspace(0.1, k_cap_max, 100)
    no_points = len(capping_const_row)
    ppr_eq_row = zeros(no_points)
    wh2_eq_row = zeros(no_points)
    ends_eq_row = zeros(no_points)

    for i in range(no_points):
        i_ppr_eq, i_wh2_eq, i_ends_eq = assemble_network(load_const = k_load,
                                                         transfer_const = k_transfer,
                                                         elongation_const = k_pol,
                                                         nucleation_const = k_nuc,
                                                         ppr_max = pa_max,
                                                         wh2_max = wa_max,
                                                         capping_const = capping_const_row[i])
        ppr_eq_row[i] = i_ppr_eq
        wh2_eq_row[i] = i_wh2_eq
        ends_eq_row[i] = i_ends_eq
    return ppr_eq_row, wh2_eq_row, ends_eq_row